In [1]:
%load_ext autoreload
%autoreload 2

import datajoint as dj
dj.config["enable_python_native_blobs"] = True
dj.config['nnfabrik.schema_name'] = "nnfabrik_monkey_mei"
schema = dj.schema("nnfabrik_monkey_mei")

import os
from os import listdir
from os.path import isfile, join


import torch
import numpy as np
import pickle 
import time
import matplotlib.pyplot as plt
import seaborn as sns

import nnfabrik
from nnfabrik import main, builder
from nnfabrik.main import *

import nnvision
from nnvision.tables.from_nnfabrik import TrainedModel
from nnvision.tables.legacy.from_mei import TrainedEnsembleModel

Connecting pawelp@134.2.168.16:3306


In [5]:
dataset_hash='a737408da40cfedd036c2eb1b8a31ff1'
ensemble_hash='e7a8d0b961ee37ffda0779bfed95f1cf'

In [6]:
model_key = (TrainedModel * TrainedEnsembleModel.Member()&dict(ensemble_hash=ensemble_hash)).fetch("KEY", limit=1, order_by="score DESC")
model_key

[{'model_fn': 'nnvision.models.se_core_point_readout',
  'model_hash': 'f8bcd882c48a55dc6cd6d7afb656f1f9',
  'dataset_fn': 'nnvision.datasets.monkey_static_loader',
  'dataset_hash': 'a737408da40cfedd036c2eb1b8a31ff1',
  'trainer_fn': 'nnvision.training.nnvision_trainer',
  'trainer_hash': '2a2816c50c67295376b4e89448430985',
  'seed': 2000,
  'ensemble_hash': 'e7a8d0b961ee37ffda0779bfed95f1cf'}]

In [7]:
model_config = TrainedModel().get_full_config(model_key, include_trainer=True)
del model_config['state_dict']
model_config.keys()

dict_keys(['model_fn', 'model_config', 'dataset_fn', 'dataset_config', 'trainer_fn', 'trainer_config'])

In [8]:
trainer = builder.get_trainer(model_config['trainer_fn'], model_config['trainer_config'])
model_config['trainer_config']

{'max_iter': 100,
 'verbose': False,
 'lr_decay_steps': 3,
 'avg_loss': False,
 'patience': 3,
 'lr_init': 0.0042}

In [9]:
dataloaders = builder.get_data(model_config['dataset_fn'], model_config['dataset_config'])
model_config['dataset_config']

{'dataset': 'CSRF19_V1',
 'neuronal_data_files': ['/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3639492658943.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3639664527524.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3639749909659.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3632669014376.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3632932714885.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3640095265572.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3639060843972.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3634658447291.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3636034866307.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3638973674012.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3634744023164.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neu

In [10]:
model = builder.get_model(model_config['model_fn'], model_config['model_config'], dataloaders=dataloaders,seed=1000)
model_config['model_config']

{'pad_input': False,
 'stack': -1,
 'depth_separable': True,
 'input_kern': 24,
 'gamma_input': 10,
 'gamma_readout': 0.5,
 'hidden_dilation': 2,
 'hidden_kern': 9,
 'se_reduction': 16,
 'n_se_blocks': 2,
 'hidden_channels': 32}

In [11]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=model_key[0]['seed'])

Epoch 1: 100%|██████████| 3616/3616 [01:01<00:00, 58.54it/s]


[001|00/03] ---> 0.27318477630615234


Epoch 2: 100%|██████████| 3616/3616 [00:50<00:00, 72.13it/s]


[002|00/03] ---> 0.3224351108074188


Epoch 3: 100%|██████████| 3616/3616 [00:50<00:00, 71.14it/s]


...

[051|01/03] -/-> 0.3985361158847809


Epoch 52: 100%|██████████| 3616/3616 [00:50<00:00, 71.54it/s]


[052|02/03] -/-> 0.3980984687805176


Epoch 53: 100%|██████████| 3616/3616 [00:54<00:00, 66.38it/s]


[053|03/03] -/-> 0.3983127772808075
Restoring best model! 0.398745 ---> 0.398745


In [12]:
score

0.39874494

In [13]:
schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__trained_model,cf9907e309d5ea8b273557321d412479,reserved,=BLOB=,,=BLOB=,kwilleke@sinz00.mlcloud.uni-tuebingen.de,6f9501d39fd2,56,60862,2020-05-07 13:33:39


In [14]:
model_config['trainer_fn']

'nnvision.training.nnvision_trainer'

In [15]:
model_config['dataset_fn']

'nnvision.datasets.monkey_static_loader'

In [16]:
model_config['model_fn']

'nnvision.models.se_core_point_readout'

## Optimal Gabor

In [17]:
import insilico_stimuli
from insilico_stimuli.stimuli import *
from insilico_stimuli.parameters import *

In [21]:
data_key = '3631807112901'
unit_idx = 27
unit_id = 26

# image dimensions
w, h = 41, 41

# Finite Set
canvas_size         = [w, h]
sizes               = FiniteParameter([float(val) for val in range(5, 31)][::5])
spatial_frequencies = FiniteParameter([float(val) for val in np.linspace(1/80, 2/5, 10)])
contrasts           = FiniteParameter([float(val) for val in np.linspace(0.0, 1.0, 5)])
orientations        = FiniteParameter([float(val) for val in np.linspace(0.0, np.pi, 10)])
phases              = FiniteParameter([float(val) for val in np.linspace(0.0, 2*np.pi, 10)]) 
grey_levels         = FiniteParameter([0.0])
eccentricities      = FiniteParameter([float(val) for val in np.linspace(0.0, 0.99, 5)])
locations           = FiniteParameter([[float(x), float(y)] for x in range(12, 28) 
                                                            for y in range(12, 28)][::16])
gabor_set = GaborSet(canvas_size = [w, h], 
                     locations = locations,
                     sizes=sizes,
                     spatial_frequencies=spatial_frequencies,
                     contrasts=contrasts,
                     orientations=orientations,
                     phases=phases,
                     grey_levels=grey_levels,
                     eccentricities=eccentricities
                    ) 

In [28]:
model(torch.zeros((32, 1, 24, 24)).cuda())

KeyError: None

In [23]:
best_params, values = gabor_set.find_optimal_stimulus(model, data_key, unit_idx)

[INFO 11-20 12:48:13] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter location_width. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 11-20 12:48:13] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter location_height. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 11-20 12:48:13] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter size. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 11-20 12:48:13] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter spatial_frequency. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool'

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same